In [2]:
import gspread
from google.oauth2.service_account import Credentials
import datetime
import pandas as pd
import numpy as np
from itertools import combinations


In [3]:
CREDENTIALS_FILENAME = "nodal-wall-416914-5876acf16f70.json"
BEO_WORKSHEET_ID = 2121169814
KIMITTUD_WORKSHEET_ID = 858480548
MASTER_SHEET_ID = "1fwhZxPtuP2MSBkTXJRSH-plXGV42mF-HlQPf4KSpFwE"

In [4]:
def initialize_gspread(service_account_file_path: str):
    credentials = Credentials.from_service_account_file(
        service_account_file_path,
        scopes=["https://www.googleapis.com/auth/spreadsheets"],
    )
    return gspread.authorize(credentials)

def sheet_to_df(sheet: gspread.Spreadsheet, worksheet_id: int | str):
    data = sheet.get_worksheet_by_id(worksheet_id).get_values()
    return pd.DataFrame(data[1:], columns=data[0])

def download_data():
    gc = initialize_gspread(CREDENTIALS_FILENAME)
    sheet = gc.open_by_key(MASTER_SHEET_ID)
    beo_df = sheet_to_df(sheet, BEO_WORKSHEET_ID)
    kimittud_df = sheet_to_df(sheet, KIMITTUD_WORKSHEET_ID)
    return beo_df, kimittud_df

In [5]:
beo_df, kimittud_df = download_data()

In [27]:
kimittud_df.columns

Index(['Hot shit?', 'HIÁNY', 'JÁTÉK', 'Co-op', 'Gyerek', 'Heavy', 'Kétfős',
       'Nyelvfüggetlen', 'Party', 'Szélproof', 'ÁDÁM', 'ALEX', 'BALU', 'BELLA',
       'BORCSA', 'BORI', 'DÁVID', 'DORINA', 'DORKA', 'EMMA', 'FANNI', 'GERGŐ',
       'JANKA', 'KATA', 'KRISTÓF', 'LILLA', 'MÁRK', 'NIKI', 'PANKA', 'RÉKA',
       'SANYI', 'SÁRI', 'TAKI', 'VANDA', 'VERONKA', 'SP', 'ROSI', 'KINGA',
       'I.ORSI', '!!!'],
      dtype='object')

In [6]:
def cleanup_beo_df(
    beo_df: pd.DataFrame,
    max_days_past: int = 100,
    day_of_event: datetime = datetime.datetime.today(),
) -> pd.DataFrame:
    filtered = beo_df.drop(beo_df.columns[[1, 2, -1, -2, -3, -4, -5]], axis=1)
    filtered.drop([0, 1, 2], inplace=True)

    day_of_event = datetime.datetime.today()
    min_date = day_of_event - datetime.timedelta(days=max_days_past)

    filtered["Dátum"] = pd.to_datetime(filtered["Dátum"], format="%Y.%m.%d.")

    filtered = filtered.loc[
        (filtered["Dátum"] > min_date) & (filtered["Dátum"] < day_of_event)
    ]

    filtered["days_before"] = (day_of_event - filtered["Dátum"]).dt.days

    filtered.set_index("Dátum", inplace=True)

    return filtered


cleaned_beo = cleanup_beo_df(beo_df)

In [56]:
def clean_kimittud_df(kimittud_df: pd.DataFrame, count_of_retired_gms: int = 4):
    df = kimittud_df.copy()
    df.drop(['Hot shit?', 'HIÁNY',  'Co-op', 'Gyerek', 'Kétfős',
       'Nyelvfüggetlen', 'Party', 'Szélproof', '!!!'], axis=1, inplace=True)
    # delete empty rows based on the last value of column 'ÁDÁM':

    column_to_check = "ÁDÁM"
    df.replace("", np.nan, inplace=True)
    df.dropna(subset=column_to_check, inplace=True)

    df.drop(df.columns[-count_of_retired_gms:], axis=1, inplace=True)

    df.drop([0,1], inplace=True)

    df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda x: x == "2")
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x == 'TRUE')
    
    df[df.columns[1:]] = df[df.columns[1:]].astype("bool")

    return df

In [57]:
kimittud = clean_kimittud_df(kimittud_df)

In [58]:
kimittud

,JÁTÉK,Heavy,ÁDÁM,ALEX,BALU,BELLA,BORCSA,BORI,DÁVID,DORINA,...,LILLA,MÁRK,NIKI,PANKA,RÉKA,SANYI,SÁRI,TAKI,VANDA,VERONKA
2,20 másodperc totál káosz,False,True,True,False,True,True,True,True,True,...,True,True,True,False,True,True,True,True,True,True
3,7 Csoda,False,False,True,True,True,True,True,True,False,...,True,False,True,True,False,False,False,True,True,False
4,7 Csoda Párbaj,False,False,True,True,True,True,False,True,False,...,True,True,True,True,False,True,False,True,True,True
5,7 Csoda: Építészek,False,False,True,False,False,False,True,True,False,...,False,True,False,False,False,False,False,True,False,True
6,8 perces birodalom,False,True,True,True,True,True,True,True,False,...,True,True,True,True,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Vörös Katedrális,True,False,True,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,True
230,Waterfall Park,False,True,True,True,False,False,True,False,False,...,True,False,False,False,False,True,False,False,False,False
231,Whirling Witchcraft,False,True,True,True,False,True,True,True,True,...,False,False,True,False,True,False,False,False,True,True
232,Wormlord,False,True,False,True,True,True,False,True,True,...,True,False,True,False,True,False,True,False,True,True


In [86]:
# input: (number of gamemasters for the event, list of gamemasters applying for an event)
def create_gm_combinations_df(
    number_of_gamemasters: int,
    list_of_applicants: list[str],
    cleaned_kimittud_df: pd.DataFrame,
    threshhold_percent: int = 57,
    heavy_threshhold_count: int = 2,
):

    dict_of_results = {}

    min_gm_count_for_game = round(number_of_gamemasters * threshhold_percent / 100)

    combination_list_of_gamemasters = list(
        combinations(list_of_applicants, number_of_gamemasters)
    )

    for comb in combination_list_of_gamemasters:

        game_counts = cleaned_kimittud_df[list(comb)].sum(axis=1)
        list_of_games_over_threshhold = (
            cleaned_kimittud_df[game_counts > min_gm_count_for_game].iloc[:, 0].tolist()
        )
        list_of_heavies_over_threshhold = (
            cleaned_kimittud_df[(game_counts >= heavy_threshhold_count) & (cleaned_kimittud_df["Heavy"] == True)].iloc[:, 0].tolist()
        )

        dict_of_results[comb] = (
            len(list_of_games_over_threshhold) + len(list_of_heavies_over_threshhold),
            list_of_games_over_threshhold,
            list_of_heavies_over_threshhold,
        )

    return pd.DataFrame.from_dict(dict_of_results, orient="index", columns=["count", "games", "heavies"])

In [87]:
list_of_applicants = [
    "DORINA",
    "BALU",
    "KATA",
    "BORI",
    "MÁRK",
    "ÁDÁM",
    "BELLA",
    "FANNI",
    "KRISTÓF",
    "RÉKA",
]

gm_combinations: pd.DataFrame = create_gm_combinations_df(6, list_of_applicants, clean_kimittud_df(kimittud_df))

In [88]:
def calc_beo_weight(days_before: int):
    if days_before == 0:
        return 1
    return np.max([0, 1 - 0.5 * np.log(days_before)])

In [89]:
def calc_beo_weights(cleaned_beo_df: pd.DataFrame):
    weights = pd.Series()

    for name, values in cleaned_beo_df.items():

        if values.dtype != np.dtypes.StrDType:

            continue

        weights[name] = cleaned_beo_df.apply(
            lambda r: calc_beo_weight(r["days_before"]) if "j" in r[name] else 0, axis=1
        ).sum()

    return weights


weights = calc_beo_weights(cleaned_beo)

In [90]:
gm_combinations['beo_weights'] = gm_combinations.apply(lambda r: weights[list(r.name)].sum(), axis=1)

In [91]:
gm_combinations.sort_values(by=['beo_weights', 'count'], ascending=[True, False]).head(10) # .to_html('gm_combinations.html')

,count,games,heavies,beo_weights
"(DORINA, BALU, KATA, MÁRK, FANNI, KRISTÓF)",130,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",0.0
"(DORINA, BALU, KATA, ÁDÁM, FANNI, KRISTÓF)",129,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",0.0
"(DORINA, BALU, KATA, MÁRK, ÁDÁM, KRISTÓF)",124,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",0.0
"(DORINA, BALU, MÁRK, ÁDÁM, FANNI, KRISTÓF)",123,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",0.0
"(DORINA, BALU, KATA, MÁRK, ÁDÁM, FANNI)",122,"[20 másodperc totál káosz, 8 perces birodalom,...","[Concordia, Ébredő Erdő, Fesztáv]",0.0
"(BALU, KATA, MÁRK, ÁDÁM, FANNI, KRISTÓF)",121,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",0.0
"(DORINA, KATA, MÁRK, ÁDÁM, FANNI, KRISTÓF)",114,"[20 másodperc totál káosz, 8 perces birodalom,...","[Ébredő Erdő, Fesztáv]",0.0
"(DORINA, BALU, KATA, BELLA, FANNI, KRISTÓF)",133,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",1.0
"(DORINA, BALU, KATA, ÁDÁM, BELLA, KRISTÓF)",132,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",1.0
"(DORINA, BALU, ÁDÁM, BELLA, FANNI, KRISTÓF)",132,"[20 másodperc totál káosz, 8 perces birodalom,...","[A Mars Terraformálása, A Mars Terraformálása:...",1.0
